In [1]:
from importlib import reload
import numpy as np

In [2]:
%cd /content/drive/MyDrive/Colab_Scratch/steps
import step
# from step import *

/content/drive/MyDrive/Colab_Scratch/steps


In [3]:
%cd /content/drive/MyDrive/Colab_Scratch/steps
reload(step)
from step import *

/content/drive/MyDrive/Colab_Scratch/steps


# [STEP 01] 상자로서의 변수

##1.2 Variable 클래스 구현

In [4]:
data = np.array(1.)
x = Variable(data)
x.data

array(1.)

In [5]:
x.data = np.array(2.)
x.data

array(2.)

# [STEP 02] 변수를 낳는 함수

##2.3 Function 클래스 이용

In [6]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x ** 2 # 구체적인 계산은 forward 메서드에서 한다
        output = Variable(y)
        return output

In [7]:
x = Variable(np.array(10.))
f = Function()

In [8]:
y = f(x)
print(type(y))

<class 'step.Variable'>


In [9]:
print(y.data)

100.0


In [10]:
f = Square()
y = f(x)

In [11]:
print(type(y))
print(y.data)

<class 'step.Variable'>
100.0


#[STEP 03] 함수 연결

In [12]:
f = Exp()

In [13]:
y = f(x)
y.data

array(22026.46579481)

In [14]:
x = Variable(np.array(0.5))
A = Square()
B = Exp()
C = Square()

In [15]:
a = A(x)
b = B(a)
c = C(b)
c.data

array(1.64872127)

# [STEP 04] 수치 미분

##4.2 수치 미분 구현

In [16]:
f = Square()
x = Variable(np.array(2.))
numerical_diff(f, x)

4.000000000004

##4.3 합성 함수의 미분

In [17]:
def f(x):
    A = Square()
    B = Exp()   
    C = Square()    
    return C(B(A(x)))

In [18]:
x = Variable(0.5)
numerical_diff(f, x)

3.2974426293330694

# [STEP 05] 역전파 이론